In [78]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools as it

In [79]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1262,tt1711425,13000000,48065672,21 & Over,Miles Teller|Skylar Astin|Justin Chon|Sarah Wr...,Scott Moore|Jon Lucas,Finally.,Brilliant student Jeff Chang has the most impo...,93,Comedy,Relativity Media|Virgin Produced|Mandeville Fi...,3/1/2013,5.7,2013
544,tt0250687,48000000,85498534,Rat Race,Cuba Gooding Jr.|Seth Green|Whoopi Goldberg|Br...,Jerry Zucker,563 miles. 9 people. $2 million. 1001 problems!,"In an ensemble film about easy money, greed, m...",112,Adventure|Comedy,Paramount Pictures|Fireworks Pictures|Alphavil...,7/30/2001,5.9,2001
1500,tt0422295,16800000,2281089,Fur: An Imaginary Portrait of Diane Arbus,Nicole Kidman|Robert Downey Jr.|Ty Burrell|Har...,Steven Shainberg,A love story.,In 1958 New York Diane Arbus is a housewife an...,122,Drama|Mystery|Romance,River Road Entertainment|Edward R. Pressman Fi...,8/30/2006,6.3,2006
321,tt1230414,85000000,219103655,It's Complicated,Meryl Streep|Alec Baldwin|Steve Martin|Lake Be...,Nancy Meyers,First comes marriage. Then comes divorce. And ...,"Ten years after their divorce, Jane and Jake A...",121,Comedy|Romance,Universal Pictures|Scott Rudin Productions|Wav...,12/23/2009,6.2,2009
471,tt1213648,25000000,4644108,London Boulevard,Keira Knightley|Colin Farrell|Anna Friel|Ray W...,William Monahan,Not every criminal wants to be one.,A parolee battles a gangster for the affection...,103,Drama|Crime,GK Films|Henceforth|Projection Pictures|London...,11/10/2010,5.5,2010


In [80]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [138]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

data['profit'] = np.subtract(data['revenue'], data['budget'])
import re
import operator
def genres_max (gen):
    genres = []
    flat_list = []
    for genre in gen:
        genres.append(genre.split("|"))

    for sublist in genres:
        for item in sublist:
            flat_list.append(item)
    genres_count = dict(Counter(flat_list))
    res = max(genres_count.items(), key=operator.itemgetter(1))[0]

    return res

def genres_profit (gen):
    genres = []
    flat_list = []
    for genre in gen['genres'][gen['profit']>0]:
        genres.append(genre.split("|"))

    for sublist in genres:
        for item in sublist:
            flat_list.append(item)
    genres_count = dict(Counter(flat_list))
    res = max(genres_count.items(), key=operator.itemgetter(1))[0]

    return res

def director_max_revenue (gen):
    max_revenue = 0
    director_max = ''
    rev_sum = 0
    for director in gen['director']:
        for rev in gen['revenue'][gen['director']==director]:
            rev_sum+=rev
        if rev_sum>max_revenue:
            max_revenue = rev_sum
            director_max = director
        rev_sum = 0


    return director_max

def director_max_action2 (data):
    dirs = []
    data_action = data[data['genres'].str.contains('Action')]
    for director in data_action['director'].str.split(pat="|"):
        dirs.append(director)

    flat_list = [item for sublist in dirs for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]

    return res

def acter_max_revenue (data):
    acters = []
    acter_max = ""
    count_rev = 0
    count_max = 0
    data_2012 = data[data['release_year'] == 2012]

    for acter in data_2012['cast'].str.split(pat="|"):
        acters.append(acter)

    flat_list = [item for sublist in acters for item in sublist]
    for acter in flat_list:
        for rev in data_2012['revenue'][data_2012['cast'].str.contains(acter)]:
            count_rev+=rev
        if count_rev > count_max:
            count_max = count_rev
            acter_max = acter
        count_rev = 0

    return acter_max

def acter_high (data):
    acters = []
    data_high = data[data['budget'] > data['budget'].mean()]

    for acter in data_high['cast'].str.split(pat="|"):
        acters.append(acter)

    flat_list = [item for sublist in acters for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]

    return res

def acter_cage (data):
    genras = []
    data_cage = data[data['cast'].str.contains("Nicolas Cage")]

    for genre in data_cage['genres'].str.split(pat="|"):
        genras.append(genre)

    flat_list = [item for sublist in genras for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]
    return res

def paramount (data):
    genras = []
    profit = 0
    profit_min = 0
    data_paramount = data[data['production_companies'].str.contains("Paramount Pictures")]

    for profit in data_paramount['profit']:
        if profit < profit_min:
            profit_min = profit
    res = data_paramount['original_title'][data_paramount['profit'] == profit_min]


    return res

def year_profit (data):
    years = []
    profits = {}

    for year in data['release_year']:
        years.append(year) if year not in years else years

    for year in years:
        for rev in data['revenue'][data['release_year'] == year]:

            if year in profits:
                profits[year] += rev

            else:
                # profits[year] = int(data['revenue'][data['release_year'] == year])
                profits[year] = rev

    dirs_count = dict(Counter(profits))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def year_profit1 (data):
    years = []
    profits = {}

    for year in data['release_year']:
        years.append(year) if year not in years else years

    for year in years:
        for rev in data['profit'][data['release_year'] == year]:

            if year in profits:
                profits[year] += rev

            else:
                # profits[year] = int(data['revenue'][data['release_year'] == year])
                profits[year] = rev

    dirs_count = dict(Counter(profits))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def month_sum (data):
    data_month = []
    for date in data['release_date']:
        data_month.append(re.search(r'\d+', date).group())

    dirs_count = dict(Counter(data_month))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def summer (data):
    count = 0
    for date in data['release_date'][data['release_date'].str.match('6|7|8')]:
        count+=1
        
    return count

def winter_director (data):
    directors = []
    for director in data['director'].str.split(pat="|")[data['release_date'].str.match('1(?!1|0)|2')]:
        directors.append(director)

    flat_list = [item for sublist in directors for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def company_title (data):
    companies = []
    companies_count = {}
    count = 0
    title_count = 0
    for company in data['production_companies'].str.split(pat="|"):
        companies.append(company) if company not in companies else companies
    flat_list = [item for sublist in companies for item in sublist]

    for company in flat_list:

        for title in data['original_title'][data['production_companies'].str.contains(company)]:
            if title is None:
                pass
            else:
                #print(title)
                title_count += 1
                for letter in title:
                    #print(letter)
                    count += 1

            #print(title_count)
            #print(count)
            companies_count[company] = round(count/title_count)
        count = 0
        title_count = 0

    #dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(companies_count.items(), key=operator.itemgetter(1))
    res = max(companies_count.items(), key=operator.itemgetter(1))[0]



    return res

def company_overview (data):
    companies = []
    companies_count = {}
    count = 0
    title_count = 0
    for company in data['production_companies'].str.split(pat="|"):
        companies.append(company) if company not in companies else companies
    flat_list = [item for sublist in companies for item in sublist]

    for company in flat_list:
        for overview in data['overview'][data['production_companies'].str.contains(company)]:
            if overview is None:
                pass
            else:
                for words in overview:
                    count += 1
                    #print(count)
                title_count += 1
            companies_count[company] = round(count/title_count)
            count = 0
            title_count = 0

    #dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(companies_count.items(), key=operator.itemgetter(1))
    res = max(companies_count.items(), key=operator.itemgetter(1))[0]



    return res

def acters_together (data):
    acters_list = []
    acters_pairs = []
    sorted_acters = []
    for acters_set in data['cast'].str.split(pat="|"):
        acters_list.append(acters_set)

    for set in acters_list:
        for subset in it.combinations(set, 2):
            acters_pairs.append(sorted(list(subset)))


    for acter_pair in acters_pairs:
        string_pairs = ""
        for ele in acter_pair :
         string_pairs += ele
        sorted_acters.append(string_pairs)




    dirs_count = dict(Counter(sorted_acters))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]
    
    return res


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [139]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '1. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [140]:
# тут пишем ваш код для решения данного вопроса:
print(data['original_title'][data['budget']==data['budget'].max()])

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


ВАРИАНТ 2

In [141]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [142]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [143]:
print(data['original_title'][data['runtime']==data['runtime'].max()])

1157    Gods and Generals
Name: original_title, dtype: object


# 3. Какой из фильмов самый короткий (в минутах)?





In [144]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [145]:
print(data['original_title'][data['runtime']==data['runtime'].min()])

768    Winnie the Pooh
Name: original_title, dtype: object


# 4. Какова средняя длительность фильмов?


In [146]:
answers['4'] = '4. 110'

In [147]:
print(data['runtime'].mean())

109.6585494970884


# 5. Каково медианное значение длительности фильмов? 

In [148]:
answers['5'] = '5. 107'

In [149]:
print(data['runtime'].median()) #5

107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [150]:
answers['6'] = '6. Avatar (tt0499549)'

In [151]:
# лучше код получения столбца profit вынести в Предобработку что в начале
print(data['original_title'][data['profit']==data['profit'].max()]) #6

239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный? 

In [152]:
answers['7'] = '7. The Lone Ranger (tt1210819)'

In [153]:
print(data['original_title'][data['profit']==data['profit'].min()]) #7

1245    The Lone Ranger
Name: original_title, dtype: object


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [154]:
answers['8'] = '8. 1478'

In [155]:
print(len(data[data['profit']>0]))

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [156]:
answers['9'] = '9. The Dark Knight (tt0468569)'

In [157]:
data2008 = data[data['release_year'] == 2008]
print(data2008['original_title'][data2008['profit'] == data2008['profit'].max()]) #9

599    The Dark Knight
Name: original_title, dtype: object


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [158]:
answers['10'] = '10. The Lone Ranger (tt1210819)'

In [159]:
data2012 = data[(data['release_year']>= 2012) & (data['release_year']<= 2014)]
print(data2012['original_title'][data2012['profit']==data2012['profit'].min()]) #10

1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?

In [160]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = '11. Drama'

In [161]:
print(genres_max(data['genres']))

Drama


ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [162]:
answers['12'] = '12. Drama'

In [163]:
print(genres_profit(data)) #12

Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [164]:
answers['13'] = '13. Peter Jackson'

In [165]:
print(director_max_revenue(data)) #13

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [166]:
answers['14'] = '14. Robert Rodriguez'

In [167]:
print(director_max_action2(data)) #14

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [168]:
answers['15'] = '15. Chris Hemsworth'

In [169]:
print(acter_max_revenue(data)) #15

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [170]:
answers['16'] = '16. Matt Damon'

In [171]:
print(acter_high(data)) #16

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [172]:
answers['17'] = '17. Action'

In [173]:
print(acter_cage(data)) #17

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [174]:
answers['18'] = '18. K-19: The Widowmaker (tt0267626)'

In [175]:
print(paramount(data)) #18

925    K-19: The Widowmaker
Name: original_title, dtype: object


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [176]:
answers['19'] = '19. 2015'

In [177]:
print(year_profit(data)) #19

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [178]:
answers['20'] = '20. 2014'

In [179]:
data_wb = data[data['production_companies'].str.contains("Warner Bros")]
print(year_profit1(data_wb)) #20

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [180]:
answers['21'] = '21. Сентябрь'

In [181]:
print(month_sum(data)) #21

9


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [182]:
answers['22'] = '22. 450'

In [183]:
print(summer(data)) #22

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [184]:
answers['23'] = '23. Peter Jackson'

In [185]:
print(winter_director(data)) #23

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [186]:
answers['24'] = '24. Four By Two Productions'

In [187]:
print(company_title(data)) #24

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [188]:
answers['25'] = '25. Midnight Picture Show'

In [189]:
print(company_overview(data)) #25

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [190]:
answers['26'] = '26. Inside Out, The Dark Knight, 12 Years a Slave'

In [191]:
data_top_one = data.nlargest(19, ['vote_average'])
print(data_top_one['original_title'])

599                                   The Dark Knight
9                                          Inside Out
34                                               Room
118                                      Interstellar
125                                The Imitation Game
119                           Guardians of the Galaxy
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
28                                          Spotlight
124                                        Big Hero 6
155                          The Theory of Everything
177                            The Fault in Our Stars
283                         

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [196]:
answers['27'] = '27. Daniel Radcliffe & Rupert Grint верно'

In [197]:
print(acters_together(data)) #27

Daniel RadcliffeRupert Grint


ВАРИАНТ 2

# Submission

In [198]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '1. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '4. 110',
 '5': '5. 107',
 '6': '6. Avatar (tt0499549)',
 '7': '7. The Lone Ranger (tt1210819)',
 '8': '8. 1478',
 '9': '9. The Dark Knight (tt0468569)',
 '10': '10. The Lone Ranger (tt1210819)',
 '11': '11. Drama',
 '12': '12. Drama',
 '13': '13. Peter Jackson',
 '14': '14. Robert Rodriguez',
 '15': '15. Chris Hemsworth',
 '16': '16. Matt Damon',
 '17': '17. Action',
 '18': '18. K-19: The Widowmaker (tt0267626)',
 '19': '19. 2015',
 '20': '20. 2014',
 '21': '21. Сентябрь',
 '22': '22. 450',
 '23': '23. Peter Jackson',
 '24': '24. Four By Two Productions',
 '25': '25. Midnight Picture Show',
 '26': '26. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '27. Daniel Radcliffe & Rupert Grint верно'}

In [199]:
# и убедиться что ни чего не пропустил)
len(answers)

27